# How to Select how many hidden layer and neurons in a neural network

### Data Preprocessing :

In [1]:
# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
df = pd.read_csv('Churn_data.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
# as we dont need the features like RowNumber	CustomerId	Surnam.
# iam dropping those features 
df = df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [4]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
#Dividing data into x and y
x = df.iloc[:,:-1]
y = df.iloc[:,-1]

#### One-hot encoding

In [6]:
#Create dummy variables 
Geography = pd.get_dummies(x["Geography"],drop_first=True)
Gender = pd.get_dummies(x["Gender"],drop_first=True)

In [7]:
#Concatenate the Data Frames
x = pd.concat([x,Geography,Gender],axis=1)
x.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0,1,0


In [8]:
#Dropping unnessary columns 
x = x.drop(["Geography","Gender"],axis=1)

In [9]:
x.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,41,1,83807.86,1,0,1,112542.58,0,1,0
2,502,42,8,159660.80,3,1,0,113931.57,0,0,0
3,699,39,1,0.00,2,0,0,93826.63,0,0,0
4,850,43,2,125510.82,1,1,1,79084.10,0,1,0


In [10]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from sklearn.preprocessing import StandardScaler

In [13]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(1)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=1)

C:\Users\kasar\AppData\Local\Temp\ipykernel_10836\281518652.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=1)


In [14]:
model

In [15]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)
#grid = GridSearchCV(estimator=model, param_grid=param_grid,cv=5)

In [16]:
grid_result = grid.fit(X_train, y_train)

Epoch 1/30
50/50 [==============================] - 1s 1ms/step - loss: 2.7668 - accuracy: 0.7980
Epoch 2/30
50/50 [==============================] - 0s 1ms/step - loss: 1.5336 - accuracy: 0.7773
Epoch 3/30
50/50 [==============================] - 0s 1ms/step - loss: 0.7648 - accuracy: 0.6934
Epoch 4/30
50/50 [==============================] - 0s 977us/step - loss: 0.6847 - accuracy: 0.6139
Epoch 5/30
50/50 [==============================] - 0s 1ms/step - loss: 0.6626 - accuracy: 0.6325
Epoch 6/30
50/50 [==============================] - 0s 777us/step - loss: 0.6378 - accuracy: 0.6591
Epoch 7/30
50/50 [==============================] - 0s 1ms/step - loss: 0.6190 - accuracy: 0.6806
Epoch 8/30
50/50 [==============================] - 0s 868us/step - loss: 0.6043 - accuracy: 0.6978
Epoch 9/30
50/50 [==============================] - 0s 1ms/step - loss: 0.5891 - accuracy: 0.7148
Epoch 10/30
50/50 [==============================] - 0s 944us/step - loss: 0.5783 - accuracy: 0.7272
Epoch 11/30

Epoch 22/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 23/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 24/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 25/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 26/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 27/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 28/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 29/30
50/50 [==============================] - 0s 2ms/step - loss: 3.2127 - accuracy: 0.7917
Epoch 30/30
13/13 [==============================] - 0s 3ms/step - loss: 2.8825 - accuracy: 0.8131
Epoch 1/30
50/50 [==============================] - 1s 3ms/step - loss: 1.6611 - accuracy: 0.7758
Epoch 2/30


50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 13/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 14/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 15/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 16/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 17/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 18/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 19/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 20/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 21/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 22/30
50/50 [===

50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 3/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 4/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 5/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 6/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 7/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 8/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 9/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 10/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 11/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1260 - accuracy: 0.7973
Epoch 12/30
50/50 [==========

50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 24/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 25/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 26/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 27/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 28/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 29/30
50/50 [==============================] - 0s 2ms/step - loss: 3.1163 - accuracy: 0.7980
Epoch 30/30
13/13 [==============================] - 0s 3ms/step - loss: 3.2682 - accuracy: 0.7881
Epoch 1/30
50/50 [==============================] - 1s 3ms/step - loss: 0.5015 - accuracy: 0.7972
Epoch 2/30
50/50 [==============================] - 0s 2ms/step - loss: 0.4798 - accuracy: 0.7972
Epoch 3/30
50/50 [======

...

25/25 [==============================] - 0s 2ms/step - loss: 2.3605 - accuracy: 0.7389
Epoch 8/30
25/25 [==============================] - 0s 2ms/step - loss: 2.2780 - accuracy: 0.7469
Epoch 9/30
25/25 [==============================] - 0s 2ms/step - loss: 2.0909 - accuracy: 0.7509
Epoch 10/30
25/25 [==============================] - 0s 2ms/step - loss: 1.9298 - accuracy: 0.7594
Epoch 11/30
25/25 [==============================] - 0s 2ms/step - loss: 1.8078 - accuracy: 0.7720
Epoch 12/30
25/25 [==============================] - 0s 2ms/step - loss: 1.7188 - accuracy: 0.7697
Epoch 13/30
25/25 [==============================] - 0s 2ms/step - loss: 1.5424 - accuracy: 0.7547
Epoch 14/30
25/25 [==============================] - 0s 2ms/step - loss: 1.4525 - accuracy: 0.7511
Epoch 15/30
25/25 [==============================] - 0s 2ms/step - loss: 1.3616 - accuracy: 0.7419
Epoch 16/30
25/25 [==============================] - 0s 2ms/step - loss: 1.2676 - accuracy: 0.7450
Epoch 17/30
25/25 [=====

25/25 [==============================] - 0s 2ms/step - loss: 0.3955 - accuracy: 0.8483
Epoch 29/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3941 - accuracy: 0.8495
Epoch 30/30
7/7 [==============================] - 0s 3ms/step - loss: 0.4581 - accuracy: 0.8356
Epoch 1/30
25/25 [==============================] - 1s 3ms/step - loss: 1.2781 - accuracy: 0.7862
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 0.7038 - accuracy: 0.7503
Epoch 3/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5981 - accuracy: 0.7867
Epoch 4/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5633 - accuracy: 0.7986
Epoch 5/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5262 - accuracy: 0.8000
Epoch 6/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5137 - accuracy: 0.7777
Epoch 7/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5060 - accuracy: 0.7953
Epoch 8/30
25/25 [=============

25/25 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8453
Epoch 19/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3996 - accuracy: 0.8478
Epoch 20/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3946 - accuracy: 0.8497
Epoch 21/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4063 - accuracy: 0.8461
Epoch 22/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3956 - accuracy: 0.8484
Epoch 23/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3886 - accuracy: 0.8487
Epoch 24/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3872 - accuracy: 0.8498
Epoch 25/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3835 - accuracy: 0.8511
Epoch 26/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3813 - accuracy: 0.8520
Epoch 27/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3794 - accuracy: 0.8531
Epoch 28/30
25/25 [===

25/25 [==============================] - 0s 2ms/step - loss: 0.4329 - accuracy: 0.8166
Epoch 9/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4265 - accuracy: 0.8205
Epoch 10/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4205 - accuracy: 0.8234
Epoch 11/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4100 - accuracy: 0.8263
Epoch 12/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4041 - accuracy: 0.8297
Epoch 13/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3991 - accuracy: 0.8306
Epoch 14/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3930 - accuracy: 0.8369
Epoch 15/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3889 - accuracy: 0.8378
Epoch 16/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3845 - accuracy: 0.8416
Epoch 17/30
25/25 [==============================] - 0s 2ms/step - loss: 0.3807 - accuracy: 0.8423
Epoch 18/30
25/25 [====

25/25 [==============================] - 0s 2ms/step - loss: 0.3457 - accuracy: 0.8622
Epoch 30/30
7/7 [==============================] - 0s 3ms/step - loss: 0.5606 - accuracy: 0.8400
Epoch 1/30
25/25 [==============================] - 1s 2ms/step - loss: 3.1484 - accuracy: 0.7958
Epoch 2/30
25/25 [==============================] - 0s 2ms/step - loss: 3.1444 - accuracy: 0.7958
Epoch 3/30
25/25 [==============================] - 0s 2ms/step - loss: 2.9913 - accuracy: 0.7958
Epoch 4/30
25/25 [==============================] - 0s 2ms/step - loss: 1.0905 - accuracy: 0.7633
Epoch 5/30
25/25 [==============================] - 0s 2ms/step - loss: 0.5409 - accuracy: 0.7873
Epoch 6/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4645 - accuracy: 0.7983
Epoch 7/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4437 - accuracy: 0.8011
Epoch 8/30
25/25 [==============================] - 0s 2ms/step - loss: 0.4342 - accuracy: 0.8053
Epoch 9/30
25/25 [==============

In [17]:
[grid_result.best_score_,grid_result.best_params_]

[0.8486250042915344,
 {'activation': 'relu',
  'batch_size': 128,
  'epochs': 30,
  'layers': [45, 30, 15]}]

In [18]:
pred_y = grid.predict(X_test)

63/63 [==============================] - 0s 2ms/step


In [19]:
y_pred = (pred_y > 0.5)

In [20]:
y_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [False],
       [False]])

In [22]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
cm

array([[1524,   71],
       [ 226,  179]], dtype=int64)

In [23]:
from sklearn.metrics import accuracy_score
score=accuracy_score(y_test,y_pred)
score

0.8515